In [1]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import torch
import utils
import test

from model import generate_model
import torch.nn as nn

import dataloader
import pandas as pd
import os
from torch.utils.data import DataLoader

In [2]:
config = utils.load_config()

In [3]:
#데이터셋 분리(Train, validation, test)
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
df_dataset = df_dataset.dropna().reset_index(drop=True)
df_oasis = df_dataset[df_dataset['source'] == 'OASIS-3']
df_adni = df_dataset[df_dataset['source'] == 'ADNI']
X_train,X_val,y_train,y_val = dataloader.dataset_split(df_oasis)
X_test = df_adni.drop(labels='group_maxinc',axis=1)
y_test = df_adni['group_maxinc']

traindata=dataloader.MRIDataset(X_train,y_train)
valdata=dataloader.MRIDataset(X_val,y_val)
testdata=dataloader.MRIDataset(X_test,y_test)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(traindata, batch_size=1, shuffle=True,num_workers=1)
val_dataloader  = DataLoader(valdata, batch_size=1, shuffle=False,num_workers=1)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

print('train_dataloader : ',len(train_dataloader.dataset))
print('val_dataloader : ',len(val_dataloader.dataset))
print('test_dataloader : ',len(test_dataloader.dataset))

train_dataloader :  659
val_dataloader :  165
test_dataloader :  166


/Users/shin/MyDir/MyGit/BrainMR_MCI/dataloader.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['grp'] = (df_dataset['source'].str.replace('OASIS-3','1').str.replace('ADNI','2').apply(pd.to_numeric)*1000


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model, _ = generate_model(model_name='resnet',model_depth = 50,n_classes=3,resnet_shortcut='B')
model.to(device)

log_path = config['log_path']
criterion_clf = nn.CrossEntropyLoss().to(device)



cpu


/Users/shin/MyDir/MyGit/BrainMR_MCI/models/resnet.py:140: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [5]:
checkpoint = torch.load('result/resnet50_checkpoint.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

In [8]:
import numpy as np
outputs = []
labels = []
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_dataloader):
        output = model(inputs)
        outputs.append(output[0].data.cpu())
        labels.append(targets[0].data.cpu())
        
outputs = torch.stack(outputs)
labels =  torch.stack(labels)

In [11]:
val_outputs = []
val_labels = []
with torch.no_grad():
    for i, (inputs, targets) in enumerate(val_dataloader):
        output = model(inputs)
        val_outputs.append(output[0].data.cpu())
        val_labels.append(targets[0].data.cpu())
        
val_outputs = torch.stack(val_outputs)
val_labels =  torch.stack(val_labels)

In [9]:
from torchmetrics.functional.classification import multiclass_auroc
multiclass_auroc(outputs, labels, num_classes=3, average="macro", thresholds=None)

tensor(0.6084)

In [12]:
multiclass_auroc(val_outputs, val_labels, num_classes=3, average="macro", thresholds=None)

tensor(0.7418)

In [10]:
from torchmetrics.classification import MulticlassConfusionMatrix
metric = MulticlassConfusionMatrix(num_classes=3)
metric(outputs, labels)

tensor([[43,  4,  5],
        [58,  7, 15],
        [14,  5, 15]])

In [13]:
metric(val_outputs, val_labels)

tensor([[94, 10,  1],
        [25,  7,  4],
        [ 7,  6, 11]])